In [20]:
from selenium import webdriver

import multiprocessing
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool, Lock, Value
import html
from time import sleep
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [21]:
def get_links(driver, url, t_sleep=1):
    result = np.array([])
    page_pass = 10000
    driver.get(url)
    sleep(t_sleep)
    items = driver.find_elements_by_class_name("gf")
    for item in items:
        result = np.append(result, item.get_attribute('data-book'))        
    while items:
        url_cur = url + '&from=' + str(result[-1])
        driver.get(url_cur)    
        sleep(t_sleep)
        items = driver.find_elements_by_class_name("gf")
        for item in items:
            result = np.append(result, item.get_attribute('data-book')) 
    return result

In [22]:
url_author = 'https://www.bookvoed.ru/author/books?id='
author_id={
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}
author_books={
  "Дарья Донцова":  np.array([]),
  "Джеймс Роллинс": np.array([]),
  "Макс Фрай":      np.array([]),
  "Эрин Хантер":    np.array([]),
  "Дмитрий Емец":   np.array([])
}

with webdriver.Chrome('C:/users/user/chromedriver.exe') as driver:
    for name in author_id:
        url = url_author+str(author_id[name])
        author_books[name] =  get_links(driver, url, 1) 
    driver.quit()    

In [7]:
def extract_book_info(book_id):
    book_info = {"ID":'', "Название":'', "Обложка":'', "Возраст":'', "Описание":'',
          "Рейтинг":'', "Понравилось":'', "В закладки":'', "Не понравилось":'', "Цена":'', 
          "Серия":'', "Издательство":'', "Год":'', "Страниц":'', "Переплёт":'', "ISBN":'', 
          "Размеры":'', "Формат":'', "Код":'', "В базе":'', "Автор":'',"Авторы":'', "Тематика":'', "Тираж":'',
          "Переводчик":'', "Производитель":''}
    
    book_info['ID']=book_id

    url_book = 'https://www.bookvoed.ru/book?id='
    html_book = requests.get(url_book+book_id)
    if html_book:
        soup = BeautifulSoup(html_book.text, 'html')

        table = soup.find('table', class_='tw')
        rows = table.find_all('tr')
        data = [list(map(lambda x: x.text, row.find_all('td'))) for row in rows]
        df = pd.read_html(str(table))[0]
        for i, a in enumerate(df[0]):
            a = a.replace(':','')
            book_info[a]=df[1][i]

        price = soup.find('div', class_='Hu Wu')
        if price:
            book_info['Цена']=price.text.split()[0]
        
        name = soup.find('h1', itemprop='name')
        if name:
            name = name.text
            p_name = soup.find('span', class_='Av')
            if p_name:
                    name = name.replace(p_name.text, '').strip()
            book_info['Название'] = name

        unlikes = soup.find('a', class_='Ke Oe ')
        if unlikes:
            book_info['Не понравилось'] = unlikes.text.strip()

        likes = soup.find('a', class_='Ke Me ')
        if likes: 
            book_info['Понравилось'] = likes.text.strip()

        bookmark = soup.find('a', class_='Ke Me ')
        if bookmark:
            book_info['В закладки'] = bookmark.text.strip()

        cover = soup.find('img', class_='tf')
        if cover:
            book_info['Обложка']=cover.attrs['src']

        description = soup.find('div', class_='lw')
        if description:
            description = description.text 
            d1 = soup.find('div', class_='kw')
            if d1:
                description = description.replace(d1.text, '')
            d2 = soup.find('div', class_='ow')
            if d2:
                description = description.replace(d2.text, '')    
            book_info['Описание'] = description.strip()

        rating = soup.find('div', class_='af')
        if rating:
            book_info['Рейтинг'] = rating.attrs['style'].split()[1]
        
        age=''
        if  soup.find('div', class_='sv nM'):
            age = '18+'
        elif soup.find('div', class_='rv nM'):
            age = '16+'
        elif soup.find('div', class_='sv nM'):
            age = '12+'
        elif soup.find('div', class_='pv nM'):
            age = '6+'
        elif soup.find('div', class_='ov nM'):
            age = '0+'    
        book_info['Возраст'] = age
    
    return book_info


In [23]:
def func_wrapper(book_id):
    res = extract_book_info(book_id)
    lock.acquire()
    global n_processed
    n_processed.value += 1
    if n_processed.value % 10 == 0:
        print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    lock.release()  
    return res

In [24]:
pool = ThreadPool(10)
lock = Lock()
n_processed = Value('i', 0)

books_id = []
for a in author_books:
    books_id = np.append(books_id, np.array(author_books[a]))
result = []
result += pool.map(func_wrapper, books_id)
pool.close()
pool.join()
pd.set_option('display.max_columns', 35)
pd.set_option('display.max_rows', 1000)
df = pd.DataFrame(result)
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)
df    

1770 objects are processed...

,ID,ISBN,Автор,Авторы,В базе,В закладки,Возраст,Год,Издательство,Код,Название,Не понравилось,Обложка,Описание,Переводчик,Переплёт,Понравилось,Производитель,Размеры,Рейтинг,Серия,Страниц,Тематика,Тираж,Формат,Цена
0,9340995,978-5-04-099550-9,Донцова Дарья Аркадьевна,,Э.ИД(м).Вечный двигатель маразма,7,16+,2019,Эксмо,1524328,Вечный двигатель маразма,1,/files/1836/59/23/24/8.jpeg,Самая опасная профессия – это писатель! А вы к...,,мягкий,7,,"10,40 см x 16,50 см x 2,10 см",71.43%,Иронический детектив (обложка),,Отечественные,40000,70x90/32,168
1,9340993,978-5-04-100203-9,Донцова Дарья Аркадьевна,,Э.ИД(м).Шоколадное пугало,3,16+,2019,Эксмо,1524330,Шоколадное пугало,,/files/1836/59/23/24/6.jpeg,"Если вы заботитесь о своем здоровье, не читайт...",,мягкий,3,,"10,40 см x 16,50 см x 2,10 см",66.67%,Иронический детектив (обложка),320,Отечественные,40000,70x90/32,168
2,9122215,978-5-04-098788-7,Донцова Дарья Аркадьевна,,Э.ИД(м).Коррида на раздевание,5,16+,2019,Эксмо,1508336,Коррида на раздевание,2,/files/1836/58/67/68/7.jpg,﻿Это прямо индийское кино какое-то! Никогда не...,,мягкий,5,,"10,50 см x 16,50 см x 2,30 см",61.54%,Иронический детектив (обложка),320,Отечественные,4000,70х90/32,167
3,9689115,978-5-04-100629-7,Донцова Дарья Аркадьевна,,Э.ИД(м).Дневник пакостей Снежинки,2,16+,2019,Эксмо,1547585,Дневник пакостей Снежинки,,/files/1836/63/96/94/0.jpeg,"Думаете, такое было только в комедии ""Ирония с...",,мягкий,2,,"10,30 см x 16,50 см x 2,00 см",62.5%,Иронический детектив,320,Отечественные,35000,70x90/32,167
4,8645468,978-5-04-096033-0,Донцова Дарья Аркадьевна,,Э.ИД(м).Последняя гастроль госпожи Удачи,12,16+,2018,Эксмо,1467227,Последняя гастроль госпожи Удачи,2,/files/1836/55/16/40/5.jpg,Дома у Даши Васильевой полный кавардак и нераз...,,мягкий,12,,"10,40 см x 16,50 см x 2,00 см",85.71%,Иронический детектив (обложка),320,Отечественные,50000,70x90/32,162
5,8835701,978-5-04-097265-4,Донцова Дарья Аркадьевна,,Э.Запасной выход из комы,6,16+,2018,Эксмо,1495047,Запасной выход из комы,3,/files/1836/56/41/53/3.jpg,"Ну кто бы мог подумать, что путешествия во вре...",,мягкий,6,,"10,40 см x 16,40 см x 2,10 см",60%,Иронический детектив (обложка),320,Отечественные,45000,70x90/32,162
6,8867666,978-5-04-098472-5,Донцова Дарья Аркадьевна,,Э.Гнездо перелетного сфинкса,8,16+,2018,Эксмо,1499165,Гнездо перелетного сфинкса,,/files/1836/58/13/59/3.jpg,"Так и хочется воскликнуть: ""Господи, избавь ме...",,мягкий,8,,"10,40 см x 16,60 см x 2,00 см",78.57%,Иронический детектив (обложка),320,Отечественные,40000,70x90/32,162
7,8505180,978-5-04-093340-2,Донцова Дарья Аркадьевна,,Э.ИД(м).Родословная до седьмого полена,18,16+,2018,Эксмо,1443191,Родословная до седьмого полена: роман,5,/files/1836/52/07/76/3.jpeg,"Ничего себе пердимонокль! Оказывается, рядом с...",,мягкий,18,,"10,50 см x 16,50 см x 0,20 см",78.26%,Иронический детектив (обложка),320,Отечественные,50000,70x90/32,161
8,9409235,978-5-04-100207-7,Донцова Дарья Аркадьевна,,Э.ИД.Леди Несовершенство,4,16+,2019,Эксмо,1532255,Леди Несовершенство,,/files/1836/59/55/94/0.jpeg,"Из чугуна его сделали, что ли? Новенький розов...",,твердый,4,,"12,00 см x 19,70 см x 2,00 см",70%,Иронический детектив,320,Отечественные,10000,80x100/32,361
9,7493745,978-5-699-99396-3,Донцова Дарья Аркадьевна,,Э.ИД(м).Ночной клуб на Лысой горе,22,16+,2017,Эксмо,1383228,Ночной клуб на Лысой горе: роман,13,/files/1836/41/01/81/8.jpg,В доме Даши Васильевой жуткий переполох – умир...,,мягкий,22,,"10,40 см x 16,50 см x 2,20 см",62.86%,Иронический детектив (обложка),352,Отечественные,55000,70х90/32,152
